In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1)
import sys
import sklearn
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
from anchor import utils
from anchor import anchor_tabular

import fatf
import fatf.utils.data.datasets as fatf_datasets

In [2]:
iris_data_dict = fatf_datasets.load_iris()
iris_data = iris_data_dict['data']
iris_target = iris_data_dict['target']
iris_feature_names = iris_data_dict['feature_names'].tolist()
iris_target_names = iris_data_dict['target_names'].tolist()



In [3]:
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(iris_data, iris_target, train_size=0.80)

In [4]:
blackbox_model = sklearn.ensemble.RandomForestClassifier(n_estimators=10)
blackbox_model.fit(train, labels_train)
print('Train', sklearn.metrics.accuracy_score(labels_train, blackbox_model.predict(train)))
print('Test', sklearn.metrics.accuracy_score(labels_test, blackbox_model.predict(test)))

Train 1.0
Test 0.9666666666666667


In [5]:
explainer = anchor_tabular.AnchorTabularExplainer(
    iris_target_names,
    iris_feature_names,
    train)

In [17]:
idx = 2
np.random.seed(1)
print('Prediction: ', explainer.class_names[int(blackbox_model.predict(test[idx].reshape(1, -1))[0])])
exp = explainer.explain_instance(test[idx], blackbox_model.predict, threshold=0.95)

Prediction:  versicolor


In [18]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())

Anchor: 1.60 < petal length (cm) <= 5.12 AND petal width (cm) <= 1.80 AND sepal length (cm) > 6.40
Precision: 0.94
Coverage: 0.06


In [8]:
exp.features()

[2, 2, 3, 0]

In [9]:
test[idx][exp.features()]

array([4.4, 4.4, 1.4, 6.6], dtype=float32)

In [10]:
# Get test examples where the anchora pplies
fit_anchor = np.where(np.all(test[:, exp.features()] == test[idx][exp.features()], axis=1))[0]
print('Anchor test precision: %.2f' % (np.mean(blackbox_model.predict(test[fit_anchor]) == blackbox_model.predict(test[idx].reshape(1, -1)))))
print('Anchor test coverage: %.2f' % (fit_anchor.shape[0] / float(test.shape[0])))

Anchor test precision: 1.00
Anchor test coverage: 0.03


In [11]:
print('Partial anchor: %s' % (' AND '.join(exp.names(1))))
print('Partial precision: %.2f' % exp.precision(1))
print('Partial coverage: %.2f' % exp.coverage(1))

Partial anchor: 1.60 < petal length (cm) <= 5.12 AND petal width (cm) <= 1.80
Partial precision: 0.73
Partial coverage: 0.45


In [12]:
fit_partial = np.where(np.all(test[:, exp.features(1)] == test[idx][exp.features(1)], axis=1))[0]
print('Partial anchor test precision: %.2f' % (np.mean(blackbox_model.predict(test[fit_partial]) == blackbox_model.predict(test[idx].reshape(1, -1)))))
print('Partial anchor test coverage: %.2f' % (fit_partial.shape[0] / float(test.shape[0])))

Partial anchor test precision: 1.00
Partial anchor test coverage: 0.07
